In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold, StratifiedShuffleSplit
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer , RobustScaler , MinMaxScaler , StandardScaler,PowerTransformer
from sklearn.metrics import roc_auc_score , roc_curve
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier 

from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
X = train.drop(['id','claim'],axis = 1)
y = train['claim']

test_data = test.drop('id',axis = 1).copy()

In [ ]:
# Credits to @hiro5299834
X['n_missing'] = X.isna().sum(axis=1)
test_data['n_missing'] = test_data.isna().sum(axis=1)

X['min'] = X.min(axis=1)
test_data['min'] = test_data.min(axis=1)

X['max'] = X.max(axis=1)
test_data['max'] = test_data.max(axis=1)

X['std'] = X.std(axis=1)
test_data['std'] = test_data.std(axis=1)

In [ ]:
imputer = SimpleImputer()

X_imputed =  imputer.fit_transform(X)
test_imputed =  imputer.transform(test_data)

In [ ]:
sc = RobustScaler()
sc.fit(X_imputed)
X_scaled = pd.DataFrame(sc.transform(X_imputed), columns=X.columns)
X_test_scaled = pd.DataFrame(sc.transform(test_imputed), columns=test_data.columns)

In [ ]:
# Thank you for Cat2 Param : https://www.kaggle.com/mlanhenke/tps-09-single-catboostclassifier-0-81676
cat2_params = {
    'iterations': 15585, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 7, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'eval_metric' : 'AUC',
    'verbose' : 500,
    'early_stopping_rounds' : 500,
}

In [ ]:
test_preds_cat = 0
train_oof = np.zeros((X_scaled.shape[0],1))
train_oof = pd.DataFrame(train_oof, columns=['preds'])
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X_scaled):
    clf = CatBoostClassifier(**cat2_params)
    trn_X, trn_y = X_scaled.loc[train_index], y.loc[train_index]
    val_X, val_y = X_scaled.loc[test_index], y.loc[test_index]
    clf.fit(trn_X,trn_y)
    oof_proba = clf.predict_proba(val_X)[:,1]
    train_oof.loc[test_index,'preds'] = oof_proba
    tmp = clf.predict_proba(X_test_scaled)
    test_preds_cat += tmp/5
    
print("The score for OOF is:")
roc_auc_score(train['claim'], train_oof['preds'])

In [ ]:
submission['claim'] = test_preds_cat[:,1]
submission.to_csv('catboost_classifier.csv', index=False)
submission.head()